# Hartree Fock Approximation: Wide Band Limit

### Structure of notebook

1. Parameters of the cattering region are defined.
2. Parameters of the fermi dirac function & Integration interval are defined.
3. Electron Density is self-consistently calculated
4. The magnetocurrent $\Delta I(m,V) = I(m) - I(-m)$ is calculated.
5. P value is calculated

## Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Path to modules

In [ ]:
import sys
sys.path.insert(0, '/Users/khhuisman/Documents/Jupyter_notebooks/Coulomb_Paper3_PHS_Collinear/Modules_Coulomb_Github')

# sys.path.insert(0, '<path_to_modules>')

## Scattering Region

In [ ]:
import Sgeom_scatteringregion as Sgeom

## Functions for electron density

In [ ]:
import handy_functions_coulomb as hfc
import Integration_HFA as Integration_method
import Currents_HF_git as Current_method
import Pvalue

In [ ]:
import negf_git as negf_method

# Energies to integrate over, Fermi Energy

In [ ]:
def plot_densities(nP_list_conv,nM_list_conv,V_list_convg,Hamiltonian0):
    dimlist = hfc.func_list_i(Hamiltonian0)
    plt.title('$n_{is}(m,V) - n_{i\overline{s}}(-m,V)$')
    n_list_total_convgM_swap = [hfc.pairwise_swap([ nM_list_conv[i][k] for k in dimlist]) for i in range(len(V_list_convg))
                              ]
    nP_list_plot =[ [nP_list_conv[i][k] for k in dimlist ] for i in range(len(V_list_convg)) ]


    plt.plot(V_list_convg,np.subtract(nP_list_plot,n_list_total_convgM_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()
    
    
def plot_densities_symmtry_relation(nP_list,nM_list,V_list,dim):
    
    '''
    Input:
        - nP_list,nM_list = electron densities as function of voltages for positive,negative magnetization
        - V_list = list of voltages
        - dim = total number of electron densities
        
    Ouput:
        Plot of "n_{is}(m,V) + n_{i\overline{s}}(-m,-V) - 1" as function of voltage
        this quantity should be zero (within numerical accuracy of integral & tol) 
    '''
    
    dimlist = [i for i in range(dim)]
    ones = [1 for i in range(dim)]
    n_list_swapM = [np.subtract(ones,hfc.pairwise_swap(nM_list[-1-i])) for i in range(len(nM_list))]
    nP_list_plot =[ [nP_list[i][k] for k in dimlist ] for i in range(len(V_list)) ]

    plt.title('$n_{is}(m,V) + n_{i\overline{s}}(-m,-V) - 1$')
    plt.plot(V_list,np.subtract(nP_list_plot,n_list_swapM))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

In [ ]:
def func_energies_alt(Hamiltonian0,U,npoints,npoints_tail):
    '''
    Input:
    Hamiltonian0 = molecule without interaction (U=0)
    U = interaction strength
    npoints = number of energy points in window [emin,emax]
    Output:
    emax = upper bound of intergral
    emin = lowest eigenvalue of Hamiltonian0
    energiesreal = list of energies between [emin-1000,emax] (make sure the emax < ef +Vmax/2)
    eflist = list of fermi energies for which to calculate electron densities.
    '''
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist))/10 + 10,2)   #lower bound for integrals
    
    
    energies_zero4000 = np.linspace(emin-5000,emin,npoints_tail) #In wide band limit the integrand has a long "tail"
   
    energiesreal_prime = np.linspace(emin,emax,npoints)
    
    energiesreal = np.unique(hfc.jointwolist(energies_zero4000,energiesreal_prime))
    
    
    

    
    return emin-5000,emin,emax,npoints,energiesreal

def func_energies(Hamiltonian0,n00listguess,
                   GammaL,GammaR,U,npoints,npoints_tail,
                   ef,T,Vmax,tol_nintegrand):
    '''
    Input:
    Hamiltonian0 = Hamiltonian of the isolated molecule without interactions (U=0)
    U            = interaction strength
    npoints      = number of energy points in window [emin,emax]
    npointstail = number of energy points in window [emin2,emin)
    Output:
        energiesreal = list of energies to calculate the Glesser integral with.
        emax = upper bound of intergral
        emin = lowest eigenvalue of Hamiltonian0 - 10 eV
        emin2 = lower bound of integral.    
    '''
    
    
    mu_max = ef + Vmax/2            ## largest chemical potential
    beta = negf_method.func_beta(T) ## beta
    
    dim = Hamiltonian0.shape[0]
    
    #Lower bound: 
    ### At lower bound of integral:
    ### <n>  = \int G+ fi Gammai G- dE ~ \int G+ Gammai G- dE
    ### the fermi functions are approximately 1.
    ### The integrand-size is related to the lowest eigenvalue of "Hamiltonian0":
    evlist = np.linalg.eigh(Hamiltonian0)[0]        ##list of eigenvalues
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1] ##lumo energy
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]   ##homo energy
    hl_gap = e_lumo - e_homo                        ##HL energy
    
    ### Therefore we 'guess' a lowest value:
    emin = np.round(int(10*min(evlist))/10 -10 ,2) 
    
    ### and check if this falls within a tolerance 'tol_nintegrand':
    emin2 = emin - 20
    boundbool = False
    HamiltonianU = negf_method.Hamiltonian_HF(n00listguess,U,Hamiltonian0)
    print('Estimating lower bound...')
    while boundbool == False:
        
        emin2 = emin2 - 30
        
        
        nlist_min    =  [ negf_method.ndensity_listi(emin2,
                                                     HamiltonianU,
                                                     GammaL,GammaR,
                                                    ef, ef,
                                                    betaL,betaR)[i] for i in range(dim)]

        check_zerob, boundbool = hfc.check_listzero(nlist_min,tol_nintegrand)
#         print(emin2,check_zerob,boundbool)
    
  
    
    
    ### Upper Bound:
    ### Due to fermi dirac function, the integrand of <n>:
    ### <n> = \int G+ fi Gammai G- dE ~ \int fi dE 
    ### will fall off quickly to zero near the energy ef + V/2
    ### Therefore the largest energy we need to consider is close to ef + Vmax/2:
    
    emax = mu_max #intial guess emax
    fmax = negf_method.fermi_dirac(emax,mu_max,beta) #intial guess efmax
    
    
    ### We continue our search iteratively
    while fmax >= tol_nintegrand/1000:
        emax = np.round(emax + 0.5,2)
        fmax = negf_method.fermi_dirac(emax,mu_max,beta)
    
    
    
    
    energies_tail = np.linspace(emin2,emin,npoints_tail)  #In wide band limit the integrand has a long "tail"
    energiesreal_prime = np.linspace( emin, emax,npoints) 
    energiesreal = np.unique(hfc.jointwolist(energies_tail,energiesreal_prime))
   
    return emin2,emin,emax,npoints,energiesreal


## Define Scattering Region

Hamiltonian0:
- Without Coulomb interactions 
- Not attached to leads.

In [ ]:
Lm = 2             #number of sites 
Wg = 2
kmax = 4

epsilon = 0 # onsite energy
t = 2.4     # hopping paramter

# List of Spin-Orbit Coupling parameter
lambdalist = [
            (1*(10**-1))*t
#             ,(1*(10**-3))*t
            ]
# List of interaction strength
Ulist = [ 
       
        1*t
        ] 


#Gamma Matrix
gammaL = 0.5    # coupling strength to left lead
pz    = 0.5    # magnetic polarization

pz_P = abs(pz)
pz_M = -abs(pz)


T = 300 # Kelvin Temperature of leads
betaL,betaR = negf_method.func_beta(T), negf_method.func_beta(T) # 1/ElectronVolt
lambda1 = lambdalist[0]



# Bias window

In [ ]:
Vmax = 4 # Maximum bias voltage [eV]
dV   = 0.1 # stepsize
V_list_pos_bias,V_list_total = hfc.func_V_total(Vmax,dV)

# Self-Consistent criteria & Integration

Comment on the convergence: Hartree Fock converges well for small U i.e. U<t.

In [ ]:
tol                 = 1*10**-4 #tolerance on the electron densities 
tol_nintegrand      = 10**-7   #cut-off for integral over Glesser function
max_iteration       = 900      #maximum number of iterations
npoints             = 15000    #npoints energy point for Glesser integral [emin,emax]
npoints_tail        = 5000     #npoints energy point for Glesser integral [emin2,emin]
alpha               = 0.1      #linear mixing paramters
npoints_current     = 15000    #number of points for Landauer-Büttiker formula
ef                  = 0        #chemical potential, always symmetricly chosen (i.e. aligned with energy around which DOS is PHS).

In [ ]:
paramlist_tot = []
factor = 1


for U in Ulist:

    gammaR = factor*gammaL
    chi = hfc.func_chi(gammaL,gammaR)
    print(chi)
    Hamiltonian0,GammaR,GammaLP,GammaLM = Sgeom.hamiltonian_multiplesites_coupled_asym(Lm,Wg, t,lambda1,gammaL,gammaL,pz,plotbool=True)




    emin2,emin,emax,npoints,energies = func_energies(Hamiltonian0,hfc.list_halves(Hamiltonian0),
                                                       GammaLP,GammaR,U,npoints,npoints_tail,
                                                       ef,T,Vmax,tol_nintegrand)
    
   
    
    print(emin2,emin,emax)
    print('ef = {},lambda1/t = {} ,U/t = {}'.format(ef,lambda1/t,U/t) )
    
    paramterslist = [npoints_current,Lm,Wg,kmax,pz,gammaL,gammaR,T,alpha,t,U,lambda1,ef,tol, Vmax,len(V_list_total),emin2,emin,emax,npoints,npoints_tail,tol_nintegrand]
    paramlist_tot.append(paramterslist)
    print(paramterslist)
#     plt.imshow(GammaR.real)
#     plt.colorbar()
#     plt.show()
#     plt.imshow(0.5*(GammaLP + GammaLM).real)
#     plt.colorbar()
#     plt.show()

    
        
        
        
        

In [ ]:
paramlist_tot = []
for U in Ulist:
    
    gammaR = gammaL
    
    
    print(gammaR,gammaL)
    
    ## Scatering region,leads created.
    Hamiltonian0,GammaR,GammaLP,GammaLM = Sgeom.hamiltonian_multiplesites_coupled_asym(Lm,Wg, t,lambda1,gammaL,gammaR,pz,plotbool=False)

    ## energies to integrate diagonal of Glesser function 
    emin2,emin,emax,npoints,energies = func_energies(Hamiltonian0,hfc.list_halves(Hamiltonian0),
                                                       GammaLP,GammaR,U,npoints,npoints_tail,
                                                       ef,T,Vmax,tol_nintegrand)
    print(emin2,emin,emax)




    print('ef = {},lambda1/t = {} ,U/t = {}'.format(ef,lambda1/t,U/t) )
    chi = np.round( (gammaR-gammaL )/(gammaL + gammaR),2).real
    print(gammaL,chi)


    paramterslist = [npoints_current,Lm,Wg,kmax,pz,gammaL,gammaR,T,alpha,t,U,lambda1,ef,tol, Vmax,len(V_list_total),emin2,emin,emax,npoints,npoints_tail,tol_nintegrand]    
    paramlist_tot.append(paramterslist)
    
    #Electron Densities are calculated for positive,negative magnetization.
    ##positive
    nP_list_total, convglistP = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                          max_iteration,
                        ef,
                        U,
                        Hamiltonian0,
                        GammaLP,GammaR, 
                        betaL, betaR,tol,energies,
                        tol_nintegrand,alpha,plot_bool=False,trackbool=False)

    ## negative
    nM_list_total, convglistM = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                          max_iteration,
                        ef,
                        U,
                        Hamiltonian0,
                        GammaLM,GammaR, 
                        betaL, betaR,tol,energies,
                        tol_nintegrand,alpha,plot_bool=False,trackbool=False)
    
    
    
 
    
    V_list_convg, nP_list_convg,nM_list_convg =  hfc.func_symmetric_converged(V_list_total,
                                                                              nP_list_total ,convglistP,
                                                                              nM_list_total, convglistP)


    plot_densities_symmtry_relation(nP_list_convg,nM_list_convg,V_list_convg,Hamiltonian0.shape[0])




    print('---- done ----')       


    print('---- calculating currents .... ----')  
    #Calculate Currents
    IP_list = Current_method.calc_I_trapz(npoints_current,
                    V_list_convg,ef,
                  Hamiltonian0 ,
                  GammaLP,GammaR,
                  U,nP_list_convg,
                  betaL,betaR)
    print('---- +M done, now -M .... ----') 
    IM_list = Current_method.calc_I_trapz(npoints_current,
                V_list_convg,ef,
              Hamiltonian0 ,
              GammaLM,GammaR,
              U,nM_list_convg,
              betaL,betaR)



    ### Magnetocurrent
    Vprime_list,PC_list = Current_method.func_MR_list(IP_list,IM_list,V_list_convg)
    dIlist = np.subtract(IP_list,IM_list) 

    plt.title('$\Delta I(m,V)$: $E_F = {}, U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
    plt.plot(V_list_convg,dIlist)
    plt.xlabel('Bias Voltage [eV]')
    plt.ylabel('Current [eV]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

    plt.title('$E_F = {}$, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
    plt.plot(Vprime_list , PC_list )
    plt.xlabel('Bias Voltage')
    plt.ylabel('$P_C$ [%]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

    plt.title('$E_F = {} $, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
    Vlist_prime,PJ_list = Pvalue.function_PvaluedI(V_list_convg,dIlist,22)
    plt.plot(Vlist_prime,PJ_list)
    plt.xlim(0,Vmax)
    plt.ylim(-1-0.1,1+0.1)
    plt.show()



    



    


In [ ]:
print(paramlist_tot)